In [1]:
import pandas as pd
import os

In [2]:
participant_file = '/Users/yasaman/UWEXP/script-input/sensors/participants.csv'
fitbit_data_path = '/Users/yasaman/UWEXP/data/fitbit-dumps/'
data_types = {"participantID" : "str",
              "date" : "str",
              "steps" : "int64"}
institution = 'UW'

In [3]:
pids = pd.read_csv(participant_file, header = None, names = ['PID'])
file_names = pids['PID'].apply(lambda x : 'PID{0:0>3}_step_daily.csv'.format(x))

In [4]:
def read_and_clean_step(path, name, types, institution):
    full_path = '{}/{}'.format(path, name)
    step = pd.read_csv(full_path, header = 0, dtype=types)
    step['date'] = pd.to_datetime(step['date'])
    step.rename(index=str, columns={'participantID': 'PID'}, inplace=True)
    step['PID'] = name[3:6]
    columns = ['institution']
    columns.extend(list(step.columns))
    step['institution'] = institution
    step = step[columns]
    return step

In [5]:
step_aggregated = []
for name in file_names:
    if name == 'PID007_step_daily.csv':
        name = 'PID007-2_step_daily.csv'
    if name == 'PID136_step_daily.csv':
        name = 'PID136-1_step_daily.csv'   
    if os.path.isfile('{}/{}'.format(fitbit_data_path, name)):
        #print('appending data for {}'.format(name))
        step = read_and_clean_step(fitbit_data_path, name, data_types, institution)
        if name == 'PID136-1_step_daily.csv':
            name = 'PID136-2_step_daily.csv'
            #print('appending data for {}'.format(name))
            step_ = read_and_clean_step(fitbit_data_path, name, data_types, institution).set_index(['PID', 'date'])
            cols = list(step.columns)
            step = step.set_index(['PID', 'date'])
            step['steps'] = step['steps'] + step_['steps']
            step.reset_index(inplace=True)
            step = step[cols]
        step_aggregated.append(step)
    else:
        print('file {} does not exist'.format(name))

file PID017_step_daily.csv does not exist
file PID018_step_daily.csv does not exist
file PID071_step_daily.csv does not exist
file PID081_step_daily.csv does not exist
file PID083_step_daily.csv does not exist
file PID100_step_daily.csv does not exist
file PID105_step_daily.csv does not exist
file PID121_step_daily.csv does not exist
file PID133_step_daily.csv does not exist
file PID135_step_daily.csv does not exist
file PID147_step_daily.csv does not exist
file PID151_step_daily.csv does not exist
file PID152_step_daily.csv does not exist
file PID177_step_daily.csv does not exist
file PID182_step_daily.csv does not exist
file PID185_step_daily.csv does not exist
file PID187_step_daily.csv does not exist
file PID192_step_daily.csv does not exist
file PID197_step_daily.csv does not exist
file PID200_step_daily.csv does not exist
file PID204_step_daily.csv does not exist
file PID207_step_daily.csv does not exist


In [6]:
step_aggregated = pd.concat(step_aggregated, axis=0)
step_aggregated.to_csv('results/step_aggregated.csv', index=False)